<a href="https://colab.research.google.com/github/victoriamuinde07/NLP-project/blob/main/index_ipynbcolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

# Define the path to your dataset folder
dataset_path = '/content/drive/My Drive/Dataset'

# List files in the folder to confirm access
os.listdir(dataset_path)


['complaints_processed.csv']